In [1]:
import tensorflow as tf
import os
import cv2
import copy
from tqdm import tqdm
import time
from math import sqrt
from skimage import io 
from PIL import Image, ImageFilter
import random
import numpy as np
from keras.preprocessing.image import ImageDataGenerator 
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
import torchvision
import torchvision.transforms as transforms
import torch

In [3]:
# Importing and Loading the data into a data frame
dataset_path_train = 'C:/Users/Calvi/Documents/Uni/Tumor/Training/'
class_names = ['glioma', 'meningioma', 'notumor', 'pituitary']

In [4]:
def batch_generator(n,batch_size):
    return [[i for i in range(0,n)[j:j+batch_size]] for j in range(0,n,batch_size)]

In [5]:
batch_size = 55

In [6]:
## Importing training
training = {}
for i in tqdm(range(len(class_names))):
    tumor_path_train = os.path.join(dataset_path_train, class_names[i])
    training[class_names[i]+'_train'] = [Image.open(os.path.join(tumor_path_train, image)) for image in os.listdir(tumor_path_train)]

100%|████████████████████████████████████████████████████████████████████████████████████| 4/4 [00:01<00:00,  2.10it/s]


### To Tensor

In [7]:
to_tensor = torchvision.transforms.Compose([transforms.PILToTensor()])
for i in tqdm(range(len(class_names))):
    n = len(training[class_names[i]+'_train'])
    for batch in batch_generator(n, batch_size):
        for j in batch:
            training[class_names[i]+'_train'][j] = to_tensor(training[class_names[i]+'_train'][j])
    print(class_names[i]," processing done!")

 25%|█████████████████████                                                               | 1/4 [00:03<00:10,  3.50s/it]

glioma  processing done!


 50%|██████████████████████████████████████████                                          | 2/4 [00:10<00:10,  5.35s/it]

meningioma  processing done!


 75%|███████████████████████████████████████████████████████████████                     | 3/4 [00:15<00:05,  5.48s/it]

notumor  processing done!


100%|████████████████████████████████████████████████████████████████████████████████████| 4/4 [00:23<00:00,  5.90s/it]

pituitary  processing done!


### 2D to 3D

In [8]:
for i in tqdm(range(len(class_names))):
    n = len(training[class_names[i]+'_train'])
    for batch in batch_generator(n, batch_size):
        for j in batch:
            if len(np.array(training[class_names[i]+'_train'][j]).shape)==2:
                training[class_names[i]+'_train'][j] = cv2.cvtColor(training[class_names[i]+'_train'][j],cv2.COLOR_GRAY2RGB)
    print(class_names[i]," processing done!")

 25%|█████████████████████                                                               | 1/4 [00:00<00:00,  4.86it/s]

glioma  processing done!


 50%|██████████████████████████████████████████                                          | 2/4 [00:00<00:00,  3.48it/s]

meningioma  processing done!


 75%|███████████████████████████████████████████████████████████████                     | 3/4 [00:00<00:00,  3.39it/s]

notumor  processing done!


100%|████████████████████████████████████████████████████████████████████████████████████| 4/4 [00:01<00:00,  3.62it/s]

pituitary  processing done!


### Resize

In [9]:
x = 0
y = 0
n = 0
for i in range(len(class_names)):
    for j in range(len(training[class_names[i]+'_train'])):
        x += training[class_names[i]+'_train'][j].shape[0]
        y += training[class_names[i]+'_train'][j].shape[1]
        n += 1
av_height = int(y/n)
av_width = int(x/n)
av = int((av_height+av_width)/2)
hav = int(sqrt((av**2)/2))

In [10]:
resize = torchvision.transforms.Compose([torchvision.transforms.Resize((av,av))])
for i in tqdm(range(len(class_names))):
    n = len(training[class_names[i]+'_train'])
    for batch in batch_generator(n, batch_size):
        for j in batch:
            training[class_names[i]+'_train'][j] = resize(training[class_names[i]+'_train'][j])
    print(class_names[i]," processing done!")

 25%|█████████████████████                                                               | 1/4 [00:06<00:18,  6.25s/it]

glioma  processing done!


 50%|██████████████████████████████████████████                                          | 2/4 [00:15<00:15,  7.99s/it]

meningioma  processing done!


 75%|███████████████████████████████████████████████████████████████                     | 3/4 [00:27<00:09,  9.88s/it]

notumor  processing done!


100%|████████████████████████████████████████████████████████████████████████████████████| 4/4 [00:37<00:00,  9.47s/it]

pituitary  processing done!


## Augmented

In [13]:
new_training = copy.deepcopy(training)

In [14]:
to_im = torchvision.transforms.ToPILImage()
horizontal_flip = torchvision.transforms.RandomHorizontalFlip(p=1)
blur = torchvision.transforms.GaussianBlur(101, sigma=(0.1, 2.0))
ran_crop = torchvision.transforms.RandomCrop((hav,hav))
rotation = torchvision.transforms.RandomRotation(degrees=(90))

### Random Crop

In [15]:
def random_crop(image):
    cropped_image = ran_crop(image)
    resized = resize(cropped_image)
    resized_image_pil = to_im(resized)
    return resized_image_pil

In [16]:
for i in tqdm(range(len(class_names))):
    n = len(new_training[class_names[i]+'_train'])
    for batch in batch_generator(n, batch_size):
        for j in batch:
            result = random_crop(new_training[class_names[i]+'_train'][j])
            new_training[class_names[i]+'_train'].append(result)
    print(class_names[i]," processing done!")
   

 25%|█████████████████████                                                               | 1/4 [00:05<00:17,  5.77s/it]

glioma  processing done!


 50%|██████████████████████████████████████████                                          | 2/4 [00:14<00:14,  7.42s/it]

meningioma  processing done!


 75%|███████████████████████████████████████████████████████████████                     | 3/4 [00:27<00:09,  9.85s/it]

notumor  processing done!


100%|████████████████████████████████████████████████████████████████████████████████████| 4/4 [00:36<00:00,  9.07s/it]

pituitary  processing done!


### Rotating

In [19]:
for i in tqdm(range(len(class_names))):
    for j in batch_generator(len(training[class_names[i]+'_train']), batch_size)[0]:
        data = training[class_names[i]+'_train'][j]
        result = rotation(data)
        new_training[class_names[i]+'_train'].append(result)
    print(class_names[i]," processing done!")

 25%|█████████████████████                                                               | 1/4 [00:00<00:02,  1.21it/s]

glioma  processing done!


 50%|██████████████████████████████████████████                                          | 2/4 [00:01<00:01,  1.24it/s]

meningioma  processing done!


 75%|███████████████████████████████████████████████████████████████                     | 3/4 [00:02<00:00,  1.25it/s]

notumor  processing done!


100%|████████████████████████████████████████████████████████████████████████████████████| 4/4 [00:03<00:00,  1.23it/s]

pituitary  processing done!


In [20]:
for i in tqdm(range(len(class_names))):
    for batch in batch_generator(len(training[class_names[i]+'_train']), batch_size):
        for j in batch:
            if type(training[class_names[i]+'_train'][j])==torch.Tensor:
                continue
            else:
                print([i,j,type(training[class_names[i]+'_train'][j])])

100%|███████████████████████████████████████████████████████████████████████████████████| 4/4 [00:00<00:00, 432.20it/s]


### Training to Images

In [21]:
for i in tqdm(range(len(class_names))):
    for batch in batch_generator(len(training[class_names[i]+'_train']), batch_size):
        for j in batch:
            training[class_names[i]+'_train'][j]=to_im(training[class_names[i]+'_train'][j])

100%|████████████████████████████████████████████████████████████████████████████████████| 4/4 [00:08<00:00,  2.02s/it]


### Flipping

In [22]:
for i in tqdm(range(len(class_names))):
    for batch in batch_generator(len(training[class_names[i]+'_train']), batch_size):
        for j in batch:
            data = training[class_names[i]+'_train'][j]
            r1 = horizontal_flip(data)
            new_training[class_names[i]+'_train'].append(r1)
    print(class_names[i]," processing done!")   

 25%|█████████████████████                                                               | 1/4 [00:00<00:00,  3.41it/s]

glioma  processing done!


 50%|██████████████████████████████████████████                                          | 2/4 [00:00<00:00,  2.81it/s]

meningioma  processing done!


 75%|███████████████████████████████████████████████████████████████                     | 3/4 [00:01<00:00,  2.11it/s]

notumor  processing done!


100%|████████████████████████████████████████████████████████████████████████████████████| 4/4 [00:01<00:00,  2.23it/s]

pituitary  processing done!


### Blur

In [23]:
for i in tqdm(range(len(class_names))):
    for batch in batch_generator(len(training[class_names[i]+'_train']), batch_size):
        for j in batch:
            blurred_im = blur(training[class_names[i]+'_train'][j])
            new_training[class_names[i]+'_train'].append(blurred_im)

100%|███████████████████████████████████████████████████████████████████████████████████| 4/4 [37:03<00:00, 555.95s/it]


### Augmented Data to Images

In [25]:
for i in tqdm(range(len(class_names))):
    for batch in batch_generator(len(new_training[class_names[i]+'_train']), batch_size):
        for j in batch:
            if type(new_training[class_names[i]+'_train'][j]) == Image.Image:
                continue
            else:
                new_training[class_names[i]+'_train'][j]=to_im(new_training[class_names[i]+'_train'][j])

100%|████████████████████████████████████████████████████████████████████████████████████| 4/4 [00:14<00:00,  3.67s/it]


## Exporting 

In [45]:
## Training data
train_path = 'C:/Users/Calvi/Documents/Preprocessed Tumor/Training/'
for i in range(len(class_names)):
    for j in range(len(new_training[class_names[i]+'_train'])):
        image_path = train_path+class_names[i]
        new_training[class_names[i]+'_train'][j].convert('RGB').save(f'{image_path}/'+str(j)+'.jpg')